In [ ]:
!python3 -m pip install google-cloud-aiplatform --upgrade

In [ ]:
from view_demo.utils import env_vars as evar

experiment_prefix = evar.EXPERIMENT_PREFIX
project_id = evar.PROJECT_ID
staging_bucket = evar.STAGING_BUCKET
location = evar.REGION
training_image_uri = evar.BASE_TRAINING_IMAGE
experiment_prefix = evar.EXPERIMENT_PREFIX
context_window = evar.CONTEXT_WINDOW
tf_serving_image = evar.TF_SERVING_IMAGE

run_id = f'context-window-{context_window}'

In [ ]:
from google.cloud import aiplatform
from datetime import datetime

# Create and experiment tag
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
experiment_id = experiment_prefix + TIMESTAMP

# Init AI Platform
aiplatform.init(
    project=project_id,
    staging_bucket=staging_bucket,
    experiment=experiment_id
)

# Define the custom training job
job = aiplatform.CustomContainerTrainingJob(
    display_name="view-training",
    container_uri=training_image_uri,
    model_serving_container_image_uri=tf_serving_image,
)


model = job.run(
    replica_count=1, 
    model_display_name="temp-prediction",
    args=[
        f'--experiment-id={experiment_id}', 
        f'--staging-bucket={staging_bucket}',
        f'--context-window={context_window}'
    ]
)
metrics_df = aiplatform.get_experiment_df(experiment_id)
#metrics_df.loc[exp_df['run_name'] == run_id]['metric.val_mae'].values[-1]

print(metrics_df)
out_model = model